# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 26/02/2024

In [ ]:
# USER: Change the path below to point to your local copy of ImageD11:

import os

home_dir = !echo $HOME
home_dir = str(home_dir[0])

# USER: You can change this location if you want

id11_code_path = os.path.join(home_dir, "Code/ImageD11")

import sys

sys.path.insert(0, id11_code_path)

In [ ]:
# import functions we need

import os
import concurrent.futures
import timeit

import matplotlib
%matplotlib widget

import h5py
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt


import ImageD11.columnfile
from ImageD11.sinograms import properties, roi_iradon
from ImageD11.blobcorrector import eiger_spatial
from ImageD11.grain import grain
from ImageD11 import cImageD11

from ImageD11.nbGui import nb_utils as utils

In [ ]:
# NOTE: For old datasets before the new directory layout structure, we don't distinguish between RAW_DATA and PROCESSED_DATA

### USER: specify your experimental directory

rawdata_path = "/home/esrf/james1997a/Data/ihma439/id11/20231211/RAW_DATA"

!ls -lrt {rawdata_path}

### USER: specify where you want your processed data to go

processed_data_root_dir = "/home/esrf/james1997a/Data/ihma439/id11/20231211/PROCESSED_DATA/James/20240226"

In [ ]:
# USER: pick a sample and a dataset you want to segment

sample = "FeAu_0p5_tR_nscope"
dataset = "top_100um"

In [ ]:
# desination of H5 files

dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")

par_path = os.path.join(processed_data_root_dir, 'Fe_refined.par')

e2dx_path = os.path.join(processed_data_root_dir, '../../CeO2/e2dx_E-08-0173_20231127.edf')
e2dy_path = os.path.join(processed_data_root_dir, '../../CeO2/e2dy_E-08-0173_20231127.edf')

In [ ]:
# Load the dataset (for motor positions, not sure why these are not in peaks)
ds = ImageD11.sinograms.dataset.load(dset_path)

In [ ]:
def read_grains(ds):
    with h5py.File(ds.grainsfile, 'r') as hin:
        grp = hin['slice_recon']
        
        raw_intensity_array = grp['intensity'][:]
        grain_labels_array = grp['labels'][:]
       
        grains_group = 'grains'
        
        grains = []
        for gid_string in tqdm(sorted(hin[grains_group].keys(), key=lambda x: int(x))):
            gg = hin[grains_group][gid_string]
            ubi = gg.attrs['ubi'][:]
            g = ImageD11.grain.grain(ubi)
            # general grain properties
            g.gid = int(gid_string)
            g.translation = gg['translation'][:]
            g.cen = gg.attrs['cen']
            # sinogram stuff
            g.ssino = gg['ssino'][:]
            g.sinoangles = gg['sinoangles'][:]
            # reconstructions
            g.og_recon = gg['og_recon'][:]
            g.recon = gg['recon'][:]
            grains.append(g)
    
    return grains, raw_intensity_array, grain_labels_array


def read_gold_grains(ds):
    ds.grainsfile_gold = os.path.join(ds.analysispath, ds.dsname + '_grains_Au.h5')
    with h5py.File(ds.grainsfile_gold, 'r') as hin:
        grp = hin['slice_recon']
        
        raw_intensity_array = grp['intensity'][:]
        grain_labels_array = grp['labels'][:]
       
        grains_group = 'grains'
        
        grains = []
        for gid_string in tqdm(sorted(hin[grains_group].keys(), key=lambda x: int(x))):
            gg = hin[grains_group][gid_string]
            ubi = gg.attrs['ubi'][:]
            g = ImageD11.grain.grain(ubi)
            # general grain properties
            g.gid = int(gid_string)
            g.translation = gg['translation'][:]
            g.cen = gg.attrs['cen']
            # sinogram stuff
            g.ssino = gg['ssino'][:]
            g.sinoangles = gg['sinoangles'][:]
            # reconstructions
            g.og_recon = gg['og_recon'][:]
            g.recon = gg['recon'][:]
            grains.append(g)
    
    return grains, raw_intensity_array, grain_labels_array

In [ ]:
grains_Fe, raw_intensity_array_Fe, grain_labels_array_Fe = read_grains(ds)
grains_Au, raw_intensity_array_Au, grain_labels_array_Au = read_gold_grains(ds)

In [ ]:
for grain in tqdm(grains_Fe):
    grain.rgb_z = utils.grain_to_rgb(grain, ax=(0,0,1),)# symmetry = Symmetry.cubic)
    grain.rgb_y = utils.grain_to_rgb(grain, ax=(0,1,0),)# symmetry = Symmetry.cubic)
    grain.rgb_x = utils.grain_to_rgb(grain, ax=(1,0,0),)# symmetry = Symmetry.cubic)
    
for grain in tqdm(grains_Au):
    grain.rgb_z = utils.grain_to_rgb(grain, ax=(0,0,1),)# symmetry = Symmetry.cubic)
    grain.rgb_y = utils.grain_to_rgb(grain, ax=(0,1,0),)# symmetry = Symmetry.cubic)
    grain.rgb_x = utils.grain_to_rgb(grain, ax=(1,0,0),)# symmetry = Symmetry.cubic)

In [ ]:
red_Fe = np.zeros_like(grains_Fe[0].recon)
grn_Fe = np.zeros_like(grains_Fe[0].recon)
blu_Fe = np.zeros_like(grains_Fe[0].recon)

for g in tqdm(grains_Fe):
    red_Fe[grain_labels_array_Fe == g.gid] = g.rgb_z[0]
    grn_Fe[grain_labels_array_Fe == g.gid] = g.rgb_z[1]
    blu_Fe[grain_labels_array_Fe == g.gid] = g.rgb_z[2]
    
rgb_Fe = np.transpose((red_Fe, grn_Fe, blu_Fe), axes=(1, 2, 0))

red_Au = np.zeros_like(grains_Au[0].recon)
grn_Au = np.zeros_like(grains_Au[0].recon)
blu_Au = np.zeros_like(grains_Au[0].recon)

for g in tqdm(grains_Au):
    red_Au[grain_labels_array_Au == g.gid] = g.rgb_z[0]
    grn_Au[grain_labels_array_Au == g.gid] = g.rgb_z[1]
    blu_Au[grain_labels_array_Au == g.gid] = g.rgb_z[2]
    
rgb_Au = np.transpose((red_Au, grn_Au, blu_Au), axes=(1, 2, 0))

In [ ]:
fig, axs = plt.subplots(1, 2, constrained_layout=True, sharex=True, sharey=True)
axs[0].imshow(rgb_Fe, origin="lower")
axs[1].imshow(rgb_Au, origin="lower")
plt.show()

In [ ]:
rgb_Au_alpha = np.append(rgb_Au, np.zeros((rgb_Au.shape[0], rgb_Au.shape[1], 1)), axis=2)

In [ ]:
rgb_Au_alpha[rgb_Au_alpha[:, :, 0] != 0, 3] = 1

In [ ]:
(raw_intensity_array_Au != 0).astype(float)

In [ ]:
fig, axs = plt.subplots(1, 2, constrained_layout=True, sharex=True, sharey=True)
axs[0].imshow(raw_intensity_array_Fe, cmap="viridis", origin="lower")
axs[0].imshow(rgb_Au_alpha, origin="lower")
axs[1].imshow(raw_intensity_array_Fe, cmap="viridis", origin="lower")
axs[0].set_title("Ni intensity map with Aus overlaid")
axs[1].set_title("Ni intensity map")
plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True, sharex=True, sharey=True)
ax.imshow(rgb_Fe, origin="lower")
ax.imshow(raw_intensity_array_Au, alpha=(raw_intensity_array_Au != 0).astype(float), cmap="grey", origin="lower")
ax.set_title("Fe IPF Z colours with white Au")
plt.show()

In [ ]:
# orientation relationship stuff

# pick a Au grain

Au_grain = grains_Au[25]

# find out where it is in the sample

fig, ax = plt.subplots(constrained_layout=True, sharex=True, sharey=True)
ax.imshow(rgb_Fe, origin="lower")
ax.imshow(Au_grain.recon, alpha=(Au_grain.recon > 0.05).astype(float), origin="lower")
ax.set_title("Ni IPF Z colours with single Au")

vr = 50

ax.set_xlim(Au_grain.translation[0]-vr, Au_grain.translation[0]+vr)
ax.set_ylim(Au_grain.translation[1]+vr, Au_grain.translation[1]-vr)
plt.show()

In [ ]:
# find Ni grain surrounding Au grain

Au_grain_pos_nearest_px = Au_grain.translation.copy()
Au_grain_pos_nearest_px = np.round(Au_grain_pos_nearest_px).astype(int)

print(Au_grain_pos_nearest_px)

Fe_grain_gid = grain_labels_array_Fe[Au_grain_pos_nearest_px[1], Au_grain_pos_nearest_px[0]].astype(int)

print(Fe_grain_gid)

In [ ]:
# confirm Ni grain selection

# rgb_Fe_masked = rgb_

fig, ax = plt.subplots(constrained_layout=True, sharex=True, sharey=True)
ax.imshow(grain_labels_array_Fe == Fe_grain_gid, cmap="grey", origin="lower")
ax.imshow(Au_grain.recon, alpha=(Au_grain.recon > 0.05).astype(float), origin="lower")
ax.set_title("Ni IPF Z colours with single Au")

vr = 50

ax.set_xlim(Au_grain.translation[0]-vr, Au_grain.translation[0]+vr)
ax.set_ylim(Au_grain.translation[1]+vr, Au_grain.translation[1]-vr)
plt.show()

In [ ]:
# we now have a Au grain and the Fe grain that it's inside

# look for orientation relationships

Fe_grain = [grain for grain in grains_Fe if grain.gid == Fe_grain_gid][0]

print(Fe_grain.U)
print(Au_grain.U)

In [ ]:
# https://doi.org/10.1016/S1005-0302(12)60169-8
# we are looking for MC Aus
# 4 different types of OR present
# A {001} Au // {001} matrix , <100> Au // <100> matrix
# B [001] Au // [310] matrix , (020) Au // (1-31) matrix
# C [110] Au // [310] matrix , (-11-1) Au // (002) matrix
# D [001] Au // [001] matrix , (-260) Au // (020) matrix

# let's check A

In [ ]:
from xfab.parameters import read_par_file

par_file_Fe = os.path.join(processed_data_root_dir, 'Fe_refined.par')
par_file_Au = os.path.join(processed_data_root_dir, 'Au.par')

pars_Fe = read_par_file(par_file_Fe)
pars_Au = read_par_file(par_file_Au)

In [ ]:
ucell_Fe = [pars_Fe.get("cell__a"), pars_Fe.get("cell__b"), pars_Fe.get("cell__c"), pars_Fe.get("cell_alpha"), pars_Fe.get("cell_beta"), pars_Fe.get("cell_gamma")]
ucell_Au = [pars_Au.get("cell__a"), pars_Au.get("cell__b"), pars_Au.get("cell__c"), pars_Au.get("cell_alpha"), pars_Au.get("cell_beta"), pars_Au.get("cell_gamma")]

In [ ]:
ucell_Au

In [ ]:

from diffpy.structure import Lattice, Structure
from orix.crystal_map import Phase
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import Miller, Vector3d

struc_Fe = Structure(lattice=Lattice(*ucell_Fe))
struc_Au = Structure(lattice=Lattice(*ucell_Au))

cubic_Fe = Phase(point_group="m-3m", structure=struc_Fe)
cubic_Au = Phase(point_group="m-3m", structure=struc_Au)

# crystal frame:

matrix_plane = Miller(hkl=[2, 2, 0], phase=cubic_Fe)
Au_plane = Miller(hkl=[0, 0, 2], phase=cubic_Au)

matrix_dir = Miller(uvw=[1, 0, 0], phase=cubic_Fe)
Au_dir = Miller(hkl=[1, 0, 0], phase=cubic_Au)

# orientations in Orix are sample-to-crystal, so we have to invert
o_Fe = Orientation.from_matrix(Fe_grain.U.T)
o_Au = Orientation.from_matrix(Au_grain.U.T)

matrix_plane_lab = ~o_Fe * matrix_plane.symmetrise(unique=True)
Au_plane_lab = ~o_Au * Au_plane.symmetrise(unique=True)

matrix_dir_lab = ~o_Fe * matrix_dir.symmetrise(unique=True)
Au_dir_lab = ~o_Au * Au_dir.symmetrise(unique=True)

In [ ]:
fig, axs = plt.subplots(1, 2, constrained_layout=True, sharex=True, sharey=True)

axs[0].imshow(grain_labels_array_Fe == Fe_grain_gid, cmap="grey", origin="lower")
axs[0].imshow(Au_grain.recon, alpha=(Au_grain.recon > 0.05).astype(float), origin="lower")
axs[0].set_title("Directions")

axs[1].imshow(grain_labels_array_Fe == Fe_grain_gid, cmap="grey", origin="lower")
axs[1].imshow(Au_grain.recon, alpha=(Au_grain.recon > 0.05).astype(float), origin="lower")
axs[1].set_title("Planes")

vr = 100

axs[0].set_xlim(Au_grain.translation[0]-vr, Au_grain.translation[0]+vr)
axs[0].set_ylim(Au_grain.translation[1]+vr, Au_grain.translation[1]-vr)

ars = 5
txs = (50*5)*1/ars

for inc in range(len(Au_dir.symmetrise(unique=True).coordinates)):
    arrow_centre = Au_grain.translation[0:2]
    axs[0].quiver(arrow_centre[0], arrow_centre[1], Au_dir_lab.coordinates[inc, 0], Au_dir_lab.coordinates[inc, 1], color="r", scale=ars)
    axs[0].annotate(np.array_str(Au_dir.symmetrise(unique=True).coordinates[inc], precision=None, suppress_small=True), arrow_centre + Au_dir_lab.coordinates[inc, 0:2]*[1, -1]*txs, ha="center", va="center", c="red")

ars = 5
txs = (50*5)*1/ars

for inc in range(len(matrix_dir.symmetrise(unique=True).coordinates)):
    arrow_centre = Au_grain.translation[0:2] + [-20, -20]
    axs[0].quiver(arrow_centre[0], arrow_centre[1], matrix_dir_lab.coordinates[inc, 0], matrix_dir_lab.coordinates[inc, 1], color="b", scale=ars)
    axs[0].annotate(np.array_str(matrix_dir.symmetrise(unique=True).coordinates[inc], precision=None, suppress_small=True),  arrow_centre + matrix_dir_lab.coordinates[inc, 0:2]*[1, -1]*txs, ha="center", va="center", c="blue")
    
    
    
ars = 5
txs = (50*5)*1/ars

for inc in range(len(Au_dir.symmetrise(unique=True).coordinates)):
    arrow_centre = Au_grain.translation[0:2]
    axs[1].quiver(arrow_centre[0], arrow_centre[1], Au_plane_lab.coordinates[inc, 0], Au_plane_lab.coordinates[inc, 1], color="r", scale=ars)
    axs[1].annotate(np.array_str(Au_plane.symmetrise(unique=True).coordinates[inc], precision=None, suppress_small=True), arrow_centre + Au_plane_lab.coordinates[inc, 0:2]*[1, -1]*txs, ha="center", va="center", c="red")

ars = 5
txs = (50*5)*1/ars

for inc in range(len(matrix_dir.symmetrise(unique=True).coordinates)):
    arrow_centre = Au_grain.translation[0:2] + [-20, -20]
    axs[1].quiver(arrow_centre[0], arrow_centre[1], matrix_plane_lab.coordinates[inc, 0], matrix_plane_lab.coordinates[inc, 1], color="b", scale=ars)
    axs[1].annotate(np.array_str(matrix_plane.symmetrise(unique=True).coordinates[inc], precision=None, suppress_small=True),  arrow_centre + matrix_plane_lab.coordinates[inc, 0:2]*[1, -1]*txs, ha="center", va="center", c="blue")


plt.show()

In [ ]:
from scipy.spatial.transform import Rotation as R
import ImageD11.sym_u
cubic = ImageD11.sym_u.cubic()
for op in cubic.group:
    v = R.from_matrix((Fe_grain.U.T @ op.T) @ Au_grain.U).as_rotvec( degrees=True)
    ang = np.linalg.norm(v)
    print(ang, v/ang)

In [ ]:
# matrix is 1
# Au is 2

# define an orientation matrix for each of the ORs
from orix.quaternion import Misorientation, Orientation

# A {001} Au // {001} matrix , <100> Au // <100> matrix

matrix_dirs = Miller(hkl=[0, 0, 2], phase=cubic_Fe)
Au_dirs = Miller(hkl=[2, 2, 0], phase=cubic_Au)

misorien_A = Misorientation.from_align_vectors(Au_dirs, matrix_dirs)

In [ ]:


# print(misorien_B)

min_misoriens = []

for Au_grain in grains_Au[:]:
    Au_grain_pos_nearest_px = Au_grain.translation.copy()
    Au_grain_pos_nearest_px = np.round(Au_grain_pos_nearest_px).astype(int)
    Fe_grain_gid = grain_labels_array_Fe[Au_grain_pos_nearest_px[1], Au_grain_pos_nearest_px[0]].astype(int)
    try:
        Fe_grain = [grain for grain in grains_Fe if grain.gid == Fe_grain_gid][0]
        o_Fe = Orientation.from_matrix(Fe_grain.U.T, symmetry=cubic_Fe.point_group)
        o_Au = Orientation.from_matrix(Au_grain.U.T, symmetry=cubic_Au.point_group)
        
        misorien_actual = Misorientation(o_Fe * (~o_Au), symmetry=(o_Au.symmetry, o_Fe.symmetry))
        misorien_actual = misorien_actual.map_into_symmetry_reduced_zone()
        
        misorien_ref = misorien_A
        misorien_ref = misorien_ref.map_into_symmetry_reduced_zone()
        
        misorien_diff = Misorientation(misorien_ref * (~misorien_actual), symmetry=misorien_actual.symmetry)
        
        misorien_diff = misorien_diff.map_into_symmetry_reduced_zone()
        
        min_misorien = np.rad2deg(misorien_diff.angle)
        
        print(Fe_grain.gid, Au_grain.gid, min_misorien)
        
        min_misoriens.append(min_misorien)
    except IndexError:
        continue

In [ ]:
np.rad2deg(misorien_actual.angle)

In [ ]:
misorien_actual.axis

In [ ]:
o_Au.to_euler("mtex")

In [ ]:
np.savetxt(os.path.join(processed_data_root_dir, 'Fe_grain_matrix.txt'), o_Fe.to_matrix()[0])

In [ ]:
np.savetxt(os.path.join(processed_data_root_dir, 'Au_grain_matrix.txt'), o_Au.to_matrix()[0])

In [ ]:
np.savetxt(os.path.join(processed_data_root_dir, 'OR_matrix.txt'), misorien_A.to_matrix()[0])

In [ ]:
fig, ax = plt.subplots()

ax.hist(np.array(min_misoriens), bins=50)
# ax.set_xlim(0,10)

plt.show()

In [ ]:
min_misoriens